In [19]:

import redis.asyncio as redis
from redis.exceptions import ResponseError
from loguru import logger
import asyncio
import pandas as pd
import ccxt.async_support as ccxt

redis_client    = redis.Redis(host='localhost', port=6379, decode_responses=True)
ex              = ccxt.bitfinex2()
await ex.load_markets()
df              = pd.DataFrame(ex.markets).T
symbols         = df.loc[(df['type']=='spot') & (df['active']) & (df['quote']=='USDT')]['symbol'].to_list()

In [51]:
async def create_consumer_group(symbol):
    stream_name = f"ohlcv:{symbol}"
    group_name  = f"ohlcv_group_{symbol.replace('/', '_')}"
    try:
        await redis_client.xgroup_create(stream_name, group_name, id="0", mkstream=True)
    except ResponseError as e:
        if "BUSYGROUP" in str(e): return
        raise e
    except Exception as e:
        logger.exception(f"Unknown exception create_consumer_group({symbol})")

async def read_from_stream(symbol):
    _s              = symbol.replace('/', '_')
    stream_name     = f"ohlcv:{symbol}"
    group_name      = f"ohlcv_group_{_s}"
    consumer_name   = f"consumer_{_s}"
    while True:
        try:
            messages = await redis_client.xreadgroup(group_name, consumer_name, {stream_name: ">"}, count=1, block=5000)
            if not messages: continue
            *_, messages = messages[0]
            await asyncio.gather(*[redis_client.xack(stream_name, group_name, msg_id)for msg_id, data in messages])
        except Exception as e:
            logger.exception(f"[{consumer_name}] Exception in live streaming")
        finally:
            await asyncio.sleep(1)

async def start_consumers(symbols):
    tasks = [asyncio.create_task(read_from_stream(symbol)) for symbol in symbols]
    await asyncio.gather(*tasks)

async def main(symbols):
    await asyncio.gather(*(create_consumer_group(s) for s in symbols))
    await start_consumers(symbols)

In [48]:
ex = ccxt.binance()
await ex.load_markets()
df = pd.DataFrame(ex.markets).T
symbols = df.loc[(df['type']=='spot') & (df['active']) & (df['quote']=='USDT')]['symbol'].to_list()

In [ ]:
await main(symbols)